# Keras for Neural Networks - Guided Example

## MLP (Multi-Layer Perceptron)

MLP Neural Networks are a set of perceptron models organized into layers, with one layer feeding into the next.

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

from keras import backend as K

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 60,000 images with 28*28 pixels (784 pixels total)
# Need 60,000 arrays (1 per image) of length 784
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values (0 ~ 255)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices (i.e. dummify class column)
# 1 column with 10 values -> 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [3]:
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 21us/step - loss: 0.4236 - acc: 0.8765 - val_loss: 0.1925 - val_acc: 0.9424
Epoch 2/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.2040 - acc: 0.9402 - val_loss: 0.1449 - val_acc: 0.9557
Epoch 3/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.1568 - acc: 0.9521 - val_loss: 0.1185 - val_acc: 0.9635
Epoch 4/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.1300 - acc: 0.9603 - val_loss: 0.1081 - val_acc: 0.9663
Epoch 5/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.1127 - acc: 0.9657 - val_loss: 0.1015 - val_acc: 0.9701
Epoch 6/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.1023 - acc: 0.9693 - val_loss: 0.0958 - val_acc: 0.9718
Epoch 7/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.0952 - acc: 0.9707 - val_loss: 0.0883 - val_acc

## Convolutional Neural Networks

Convolution takes your data and creates overlapping subsegments testing for a given feature in a set of spaces and upon which it develops its model.

First, you have to define a shape of your input data. This can theoretically be in any number of dimensions, though for our image example we will use 2d, since images are in two dimensions. 

Over that shaped data, we then create our tiles, also called __kernels__. These kernels are like little windows, that will look over subsets of the data of a given size. In the example below we create 3x3 kernels, which run overlapping over the whole 28x28 input looking for features. That is the convolutional layer, a way of searching for a subpattern over the whole of the image. 

Next comes a pooling layer. This is a downsampling technique, which effectively serves to reduce sample size and simplify later processes. For each value generated by our convolutional layers, it looks over the grid in non-overlapping segments and takes the maximum value of those outputs. It's not the features' exact location that matters, but its approximate or relative location. After pooling, you will flatten the data back out, so  it can be put into dense layers as in MLP.

In [5]:
# img_rows, img_cols = 28, 28
# num_classes = 10

# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# # Building the Model
# model = Sequential()
# # First convolutional layer, note the specification of shape
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=128,
#           epochs=10,
#           verbose=1,
#           validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

## Hierarchical Recurrrent Neural Networks

So far when we've talked about neural networks we've talked about them as feedforward. Recurrent neural networks do not obey that directional logic, instead letting the data cycle through the network.

You have to use recurrent layers and often time distribution (which handles the extra dimension created through the LTSM layer, as a time dimension) to get these things running. You can find an example of a hierarchical recurrent network below (via the link here). When you get comfortable with networks as they exist in Keras for both convolution and MLP, start exploring recurrence.

In [6]:
# batch_size = 64
# num_classes = 10
# epochs = 3

# # Embedding dimensions.
# row_hidden = 32
# col_hidden = 32

# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# # Reshapes data to 4D for Hierarchical RNN.
# x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
# x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # Converts class vectors to binary class matrices.
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# row, col, pixel = x_train.shape[1:]

# # 4D input.
# x = Input(shape=(row, col, pixel))

# # Encodes a row of pixels using TimeDistributed Wrapper.
# encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# # Encodes columns of encoded rows.
# encoded_columns = LSTM(col_hidden)(encoded_rows)

# # Final predictions and model
# prediction = Dense(num_classes, activation='softmax')(encoded_columns)
# model = Model(x, prediction)
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

# scores = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

## Drill: 99% MLP

We have the MLP above, which runs reasonably quickly. Copy that code down here and see if you can tune it to achieve 99% accuracy with a Multi-Layer Perceptron. Does it run faster than the recurrent or convolutional neural nets?

In [35]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [48]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dropout(0.05))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 64)                50240     
_________________________________________________________________
dropout_65 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_82 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_66 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_83 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_67 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_84 (Dense)             (None, 10)                650       
Total para

In [51]:
history = model.fit(x_train, y_train, batch_size=30, epochs=10, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 66us/step - loss: 0.0818 - acc: 0.9830 - val_loss: 0.1421 - val_acc: 0.9758
Epoch 2/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.0813 - acc: 0.9827 - val_loss: 0.1615 - val_acc: 0.9761
Epoch 3/10
60000/60000 [==============================] - 4s 71us/step - loss: 0.0873 - acc: 0.9819 - val_loss: 0.1690 - val_acc: 0.9704
Epoch 4/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0875 - acc: 0.9815 - val_loss: 0.1360 - val_acc: 0.9759
Epoch 5/10
60000/60000 [==============================] - 4s 69us/step - loss: 0.0899 - acc: 0.9817 - val_loss: 0.1744 - val_acc: 0.9735
Epoch 6/10
60000/60000 [==============================] - 4s 64us/step - loss: 0.1002 - acc: 0.9813 - val_loss: 0.1532 - val_acc: 0.9767
Epoch 7/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.1002 - acc: 0.9812 - val_loss: 0.1545 - val_acc